In [1]:
import numpy as np
from numpy import random
from scipy.stats import norm
import torch
import torch.nn as nn
#import torch.nn.functional as F
# from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import TensorDataset, DataLoader, random_split
import math
from argparse import Namespace
import sys
from tqdm.notebook import tqdm
import gc

import pandas as pd
import traceback
import os  
from joblib import Parallel, delayed
#sys.path.append("/home/yuedu/deep-my")
# sys.path.append("/work/home/ac7hjj2nfu/project/indtest")
#sys.path.append("C:\\Users\\duyue\\.spyder-py3")


#from fun_all import generate_one_minus_one_distribution

import itertools
import time 
import fun_all
import gene_data_Copy1
import non_connect_nn_l1
import multiprocessing as mp
import matplotlib.pyplot as plt

In [2]:
import traceback
import smtplib
import email.utils
from email.mime.text import MIMEText
def send_email(subject, content):
    message = MIMEText(content)
    message['To'] = email.utils.formataddr(('Duuuuu', '3013081232@qq.com'))
    message['From'] = email.utils.formataddr(('Duuuuu', '3013081232@qq.com'))
    message['Subject'] = subject
    server = smtplib.SMTP_SSL('smtp.qq.com', 465)
    server.login('3013081232@qq.com', 'ysvbwhdzhtkzdfcc')
    try:
        server.sendmail('3013081232@qq.com', ['3013081232@qq.com'], msg=message.as_string())
        server.quit()
        print("邮件发送成功")
    except Exception as e:
        print("邮件发送失败:", e)

In [3]:
import logging

def init_logging(log_file):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    
    # 文件处理器
    file_handler = logging.FileHandler(log_file, mode='a', encoding='utf-8')
    file_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(message)s')
    file_handler.setFormatter(formatter)
    
    # 控制台处理器
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(formatter)
    
    # 添加处理器
    if not logger.handlers:
        logger.addHandler(file_handler)
        logger.addHandler(console_handler)
    
    return logger

In [4]:
def threshold(mat, thresh):
    mat = torch.clamp(mat, min=-thresh, max=thresh)
    return mat
def trainer(i,kk,exp_v, res_v, n_1, n_2,n, batchsize, val_pct, lr, n_epochs, patience, input_features, hidden_features1, hidden_features2, output_features, device, thresh=None, plot=False):
    # torch.manual_seed((kk + 1) *12345)
    logger = init_logging("logex6_nn.log")  # 为每个任务初始化日志
    criterion = nn.MSELoss()
    model = non_connect_nn_l1.AdaptiveNet(input_features, hidden_features1, hidden_features2, output_features).to(device)
    # optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    optimizer = torch.optim.Adam(model.parameters(),
                                   lr=lr,
                                   weight_decay=0.)
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
    #                                           step_size=20,
    #                                           gamma=0.9)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, min_lr=1e-4)
    # 定义优化器，这里使用Adam优化器

    #dataset = TensorDataset(exp_v[n_1:(n_1+n_2),:], res_v[n_1:(n_1+n_2)])
    train_dataset = TensorDataset(exp_v[n_1:(n_1+n_2),:], res_v[n_1:(n_1+n_2)])
    
    n1_ind = torch.arange(0, n_1)
    n2_ind = torch.arange(n_1+n_2, n)
    all_indices = torch.cat([n1_ind, n2_ind])
    val_dataset = TensorDataset(exp_v[all_indices,:], res_v[all_indices])
    # val_size = int(val_pct * len(dataset))
    # train_size = len(dataset) - val_size
    # train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True,drop_last=False)
    val_loader = DataLoader(val_dataset, batch_size=batchsize, shuffle=False)
    run_loss = 0 

    min_val_loss = float('inf')  # 保存最小验证损失
    no_improvement_epochs = 0  # 记录验证损失未改善的次数
    
    val_losses = [] 
    run_losses = []
    for epoch in range(n_epochs):
        model.train()
        run_loss1=run_loss
        run_loss = 0
    
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.view(-1), labels)
            # print(outputs.squeeze().shape, labels.shape)
            loss.backward()
            optimizer.step()
            run_loss += loss.item()
    
        run_loss =  run_loss / len(train_loader)
        run_losses.append(run_loss)
        #print(f"the {epoch+1}, TRAIN loss {run_loss}")
    
         #验证
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs = val_inputs.to(device)
                val_labels = val_labels.to(device)
    
                val_outputs = model(val_inputs)
                val_loss += criterion(val_outputs.view(-1), val_labels).item()
        val_loss = val_loss / len(val_loader)
        val_losses.append(val_loss)
        scheduler.step(val_loss)

        if val_loss < min_val_loss:
            min_val_loss = val_loss
            no_improvement_epochs = 0  # 重置
            best_model_state = {k: v.clone() for k, v in model.state_dict().items()}
            # torch.save(model_all.state_dict(), 'best_model.pth')  # 保存最佳模型
        else:
            no_improvement_epochs += 1

        if no_improvement_epochs >= patience or min_val_loss<1e-3:
            #print(f"Early stopping at epoch {epoch + 1}, validation loss did not improve for {patience} epochs. {min_val_loss}")
            break
    
    model.load_state_dict(best_model_state)
    if thresh:
        with torch.no_grad():
            model.eval()
            fit = threshold(model.forward(exp_v.to(device)),thresh)
    else:
        with torch.no_grad():
            model.eval()
            fit = model(exp_v.to(device))
    error = res_v.to(device) - fit.squeeze()
    if i % 10 == 0 and kk%10==0:
        logger.info(f"第{kk}次的{i}网络完成")  # 写入日志
    if plot:
        fig = plt.figure(figsize=(10, 4))
        plt.style.use('fivethirtyeight')
        plt.plot(run_losses,
                 label='Training Loss', color='C0', linewidth=2)
        plt.plot(val_losses,
                 label='Validation Loss', color='C1', linewidth=2)
        plt.yscale('log')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.tight_layout()
        plt.show()
    return {'model': model,
                'error': error,
                'losses': run_losses,
                'val_losses': val_losses,}

In [5]:
def gauss_fun(kk,x, y, z, device):
    # torch.manual_seed((kk + 1) *12345)
    n = x.shape[0]
    print(n)
    n_1 = int(n/3)
    n_2 = int(n/2)
    n_3 = n-n_1-n_2
    n_tilde = torch.arange(int(n_3/3), n_3, 2)
    p = x.shape[1]
    q = y.shape[1]
    m = z.shape[1]
    alpha=0.05


    # 实例化网络
    input_features = m
    hidden_features1 = int(128)
    hidden_features2 = int(32)
    output_features = 1
    n_epochs = 400
    patience = 30
    batchsize = 32
    val_pct = 0.15
    lr = 0.01


    x = fun_all.data_transform(x[0:n_1,:], x)
    y = fun_all.data_transform(y[0:n_1,:], y)
    z = fun_all.data_transform(z[0:n_1,:], z)
    sA = torch.zeros(3, 4)
    rep_xy = torch.cat([x, y], dim=1)
    thresh = (math.log(m*n))**(1/2) * math.log(n)
    results = [trainer(i,kk,z, rep_xy[:, i], n_1, n_2, n, batchsize, val_pct, lr, n_epochs, patience, input_features, hidden_features1, hidden_features2, output_features, device,thresh) for i in range(p + q)]
    error_all = torch.stack([res['error'] for res in results],dim=1)
    eps_all = error_all[:, 0:p]
    delta_all = error_all[:, p:(p+q)]
    
    ell = n_tilde.shape[0]
    N=5000
    N_1 = 500
    res1_red = torch.zeros(N_1, ell)
    res1_norm = torch.zeros(N_1, ell)
    res1_mm = torch.zeros(N_1, ell)
    ########data driven
    for t in range(N_1):
        # torch.manual_seed((t+1)*54321)
        f1 = (torch.randn(n, n) / (n**(1/2))).to(device)
        f2 = (torch.randn(n, n) / (n**(1/2))).to(device)
        f3 = (torch.randn(n, n) / (n**(1/2))).to(device)
        w_hs_norm = f1 @ z.to(device)
        eps_norm = f2 @ eps_all
        delta_norm = f3 @ delta_all
        with torch.no_grad():
            f_h_hs = torch.stack([res['model'].forward(w_hs_norm.to(device)).squeeze() for res in results],dim=1)
        thresh1 = norm.ppf((1-1/n_1), 0, 1)
        w_hs_norm  = threshold(w_hs_norm, thresh1)
        u_hs_norm = f_h_hs[:, 0: p] + eps_norm
        u_hs_norm = threshold(u_hs_norm, thresh1)
        v_hs_norm = f_h_hs[:, p: (p+q)] + delta_norm
        v_hs_norm = threshold(v_hs_norm, thresh1)
        if t==0:
            rep_uv = torch.cat([u_hs_norm, v_hs_norm], dim=1)
            results2 = [trainer(k,t,w_hs_norm, rep_uv[:, k], n_1, n_2, n, batchsize, val_pct, lr, n_epochs, patience, input_features, hidden_features1, hidden_features2, output_features, device,thresh) for k in range(p + q)]
        ###内部训练神经网络
        with torch.no_grad():
            f_hhm = torch.stack([res['model'].forward(w_hs_norm.to(device)).squeeze() for res in results2],dim=1)
        error_all_u = torch.cat([u_hs_norm, v_hs_norm], dim=1) - f_hhm
        eps_hhs = error_all_u[(n_1+n_2):n, 0:p]
        delta_hhs = error_all_u[(n_1+n_2):n, p:(p+q)]    
        M = fun_all.generate_one_minus_one_distribution(N, n_3).float().to(device)
        M_norm = torch.randn(N, n_3).to(device) 
        M_mm = fun_all.generate_mammen(N,n_3).float().to(device)
        for i in range(ell):
            ###critical value
            n_tilde1 = n_tilde[i]
            sqrt_n_tilde1 = torch.sqrt(torch.as_tensor(n_tilde1)).to(device)
            dat_new = torch.empty(N, p).to(device)
            dat_new_n = torch.empty(N, p).to(device)
            dat_new_mm = torch.empty(N, p).to(device)
            for j in range(p):
                Ai = eps_hhs[0:n_tilde1, j:j+1]
                dat_i = Ai * delta_hhs[0:n_tilde1, :]  # Hadamard 内积，形状 (n_tilde1, q)
                dat_i = dat_i - torch.mean(dat_i, dim=0, keepdim=True)  # 减去均值
                result_i = torch.max(abs(M[:, 0:n_tilde1] @ dat_i / sqrt_n_tilde1), axis=1)[0]  # 形状 (N,)
                dat_new[:, j] = result_i  # 存储结果
                result_i_n = torch.max(abs(M_norm[:, 0:n_tilde1] @ dat_i / sqrt_n_tilde1), axis=1)[0]  # 形状 (N,)
                dat_new_n[:, j] = result_i_n  # 存储结果
                result_i_mm = torch.max(abs(M_mm[:, 0:n_tilde1] @ dat_i / sqrt_n_tilde1), axis=1)[0]  # 形状 (N,)
                dat_new_mm[:, j] = result_i_mm  # 存储结果
            dat_new = torch.max(dat_new, axis = 1).values
            dat_new_n = torch.max(dat_new_n, axis = 1).values
            dat_new_mm = torch.max(dat_new_mm, axis = 1).values
            # dat = torch.einsum('np,nq->npq', eps_hhs[0:n_tilde1, :], delta_hhs[0:n_tilde1, :]).reshape(n_tilde1, -1)  # (n_tilde1, p * q)
            # # dat = eps_hhs[0:n_tilde1, :].repeat_interleave(q, dim=1) * delta_hhs[0:n_tilde1, :].repeat(1, p)
            # dat = dat - torch.mean(dat, dim=0)
            # #rademacher 
            # dat_new = torch.max(abs(M[:,0:n_tilde1].float() @ dat / np.sqrt(n_tilde1)), axis=1)[0]
            cv_red = torch.kthvalue(dat_new, (N-int(N*alpha)+1)).values
                #norm
            # dat_new_n = torch.max(abs(M_norm[:,0:n_tilde1] @ dat / np.sqrt(n_tilde1)), axis=1)[0]
            cv_norm = torch.kthvalue(dat_new_n, (N-int(N*alpha)+1)).values
            #norm
            # dat_new_mm = torch.max(abs(M_mm[:,0:n_tilde1].float() @ dat / np.sqrt(n_tilde1)), axis=1)[0]
            cv_mm = torch.kthvalue(dat_new_mm, (N-int(N*alpha)+1)).values
            #test-statistics
            test1 = np.sqrt(n_tilde1) * torch.max(abs(eps_hhs[0:n_tilde1, :].T @ delta_hhs[0:n_tilde1, :] / n_tilde1))
            res1_red[t, i]  = test1 >cv_red
            res1_norm[t, i] = test1 >cv_norm
            res1_mm[t, i]   = test1 >cv_mm
            #if t == 0 and i == 0:
                #print(test1, cv_red, cv_norm, cv_mm)
        #print(t)
    et = time.time()
    # print(et-st)     
    ##########

    #st2 =time.time()            
    N=5000
    n_s = n_1 + n_2

    size_red  = torch.mean(res1_red, dim=0) 
    index_red = int(max(torch.where(abs(size_red-alpha) == min(abs(size_red-alpha)))[0]))
    n_tilde_red = int(n_tilde[index_red])
    eps_t1 = eps_all[n_s:(n_s+n_tilde_red), :]
    delta_t1 = delta_all[n_s:(n_s+n_tilde_red), :]
    test_1 = torch.max(abs(eps_t1.T @ delta_t1 / np.sqrt(n_tilde_red) ))
    
    #############
    dat_1 = eps_t1.repeat_interleave(q, dim=1) * delta_t1.repeat(1, p)
    dat_1 = dat_1 - torch.mean(dat_1, dim=0)
    #rademacher
    M = fun_all.generate_one_minus_one_distribution(N,n_tilde_red).float().to(device)
    dat_new1 = torch.max(abs(M @ dat_1 / np.sqrt(n_tilde_red)), axis=1)[0]
    #cv_red = torch.kthvalue(dat_new1, (N-int(N*alpha)+1)).values
    
    sA[0,0] = torch.mean( (dat_new1 > test_1).float() )
    sA[0,1] = size_red[index_red] 
    sA[0,2]= n_tilde[index_red]
    
    size_norm  = torch.mean(res1_norm, dim=0) 
    index_norm = int(max(torch.where(abs(size_norm-alpha) == min(abs(size_norm-alpha)))[0]))
    n_tilde_norm = int(n_tilde[index_norm])
    #norm 
    M_norm = fun_all.torch.randn(N, n_tilde_norm).to(device)
    #st1 =time.time()
    eps_t2 = eps_all[n_s:(n_s+n_tilde_norm), :]
    delta_t2 = delta_all[n_s:(n_s+n_tilde_norm), :]
    test_2 =  torch.max(abs(eps_t2.T @ delta_t2 / np.sqrt(n_tilde_norm) ))
    #############
    dat_2 = eps_t2.repeat_interleave(q, dim=1) * delta_t2.repeat(1, p)
    dat_2 = dat_2 - torch.mean(dat_2, dim=0)
    dat_new2 = torch.max(abs(M_norm @ dat_2 / np.sqrt(n_tilde_norm)), axis=1)[0]
    #cv_norm = torch.kthvalue(dat_new2, (N-int(N*alpha)+1)).values
    #et1 =time.time()
    #print(et1-st1)
    sA[1,0] = torch.mean( (dat_new2 > test_2).float() ) #(test_2 > cv_norm)
    sA[1,1] = size_norm[index_norm] 
    sA[1,2]= n_tilde[index_norm]
     
    
    ####mannen       
    size_mm  = torch.mean(res1_mm, dim=0) 
    index_mm = int(max(torch.where(abs(size_mm-alpha) == min(abs(size_mm-alpha)))[0]))
    n_tilde_mm = int(n_tilde[index_mm])
    eps_t3 = eps_all[n_s:(n_s+n_tilde_mm), :]
    delta_t3 = delta_all[n_s:(n_s+n_tilde_mm), :]
    test_3 = torch.max(abs(eps_t3.T @ delta_t3 / np.sqrt(n_tilde_mm) ))
    #############
    dat_3 = eps_t3.repeat_interleave(q, dim=1) * delta_t3.repeat(1, p)
    dat_3 = dat_3 - torch.mean(dat_3, dim=0)
    #norm 
    M_mm = fun_all.generate_mammen(N,n_tilde_mm).float().to(device)
    dat_new3 = torch.max(abs(M_mm @ dat_3 / np.sqrt(n_tilde_mm)), axis=1)[0]
    #cv_mm = torch.kthvalue(dat_new3, (N-int(N*alpha)+1)).values
    sA[2,0] = torch.mean( (dat_new3 > test_3).float() ) #test_3 > cv_mm)
    sA[2,1] = size_norm[index_mm] 
    sA[2,2]= n_tilde[index_mm] 
 
    sA[:, 3] = kk
    return  sA 

In [6]:
args = Namespace(
    lr = 0.01,
    device = 'cuda'
)

In [7]:
# p=1600 
# q=1600 
# m=5 
# # n = 100 
# n_1 = int(n/3)
# n_2 = int(n/2)
# n_3 = n - n_1 -n_2

# #####n50
# n_tilde = torch.arange(int(n_3/3), n_3, 2)
alpha=0.05


# n_my = 400

dep=0
device = 'cuda:0'

In [8]:
# # 实例化网络
# input_features = m
# hidden_features1 = int(128)
# hidden_features2 = int(32)
# output_features = 1
# n_epochs = 400
# patience = 30
# batchsize = 16
# val_pct = 0.15
# lr = 0.01

In [9]:
name1 = ["CS" , "CD", "CSt" , "Eng"  ,"Fin" ,"HC" , "Ind" , "IT" , "Mat" , "RE" , "Uti"]
combi_name1 = list(itertools.combinations(name1, 2))
logger = init_logging("realdata.log")  # 为每个任务初始化日志

shape = torch.zeros(11)

title_z = '02return_' + name1[0] + '.csv'
# z = torch.from_numpy(pd.read_csv(title_z))
data = torch.from_numpy(pd.read_csv(title_z).iloc[:, 1:].to_numpy())
shape[0] = data.shape[1]
for t in range(10):
        title_zz = '02return_' + name1[t+1] + '.csv'
        zz = torch.from_numpy(pd.read_csv(title_zz).iloc[:, 1:].to_numpy())
        shape[t+1] = zz.shape[1]
        data = torch.cat((data, zz), dim=1)

combinations = list(itertools.combinations(range(11), 2))
print(shape)


tensor([21., 63., 30., 23., 63., 58., 73., 73., 23., 29., 29.])


In [10]:
stacked_tensor = torch.zeros(55, data.shape[0], data.shape[1])
for i in range(55):
    mask = torch.ones(data.shape[1], dtype=torch.bool)
    index = combinations[i]
    # print(index)
    indx1 = int( torch.sum(shape[:index[0]]) )
    indx2 = int( torch.sum(shape[:index[0]+1]) )
    indy1 = int( torch.sum(shape[:index[1]]) )
    indy2 = int( torch.sum(shape[:index[1]+1]) )
    x = data[:, int(indx1):int(indx2)] 
    #print(x.shape)
    y = data[:, int(indy1):int(indy2)] 
    # print(y.shape)
    mask[indx1:indx2] = False  # 将 x 的范围设置为 False
    mask[indy1:indy2] = False  # 将 y 的范围设置为 False
    z = data[:, mask]  # 使用布尔掩码提取 z
    # print(z.shape)
    dat = torch.cat((x, y, z), dim=1)
    stacked_tensor[i] = dat
    # print(i)


In [11]:
# name1 = ["CS" , "CD", "CSt" , "Eng"  ,"Fin" ,"HC" , "Ind" , "IT" , "Mat" , "RE" , "Uti"]
# combi_name1 = list(itertools.combinations(name1, 2))

# result = torch.zeros(55,3)
# result_size = torch.zeros(55,3)
# result_n = torch.zeros(55,3)
# for i in range(55):
#     x_name =  combi_name1[i][0] 
#     y_name =  combi_name1[i][1] 
#     z_name =  [name for name in name1 if name not in combi_name1[i]]
#     print(z_name)
#     title_x = '02return_' + x_name + '.csv' 
#     title_y = '02return_' + y_name + '.csv' 
#     x = torch.from_numpy(pd.read_csv(title_x).iloc[:, 1:].to_numpy())  
#     y = torch.from_numpy(pd.read_csv(title_y).iloc[:, 1:].to_numpy())
#     print(x.shape)
#     print(y.shape)
#     title_z = '02return_' + z_name[0] + '.csv'
#     z = torch.from_numpy(pd.read_csv(title_z).iloc[:, 1:].to_numpy())
#     for t in range(8):
#         title_zz = '02return_' + z_name[t+1] + '.csv'
#         zz = torch.from_numpy(pd.read_csv(title_zz).iloc[:, 1:].to_numpy())
#         z = torch.cat((z,zz), dim =1)
#         #print(title_zz )
#     print(z.shape)
#     print(torch.cat((x, y, z), dim=1) - stacked_tensor[i])
    
    
   


In [12]:

def run_simulation(i, stacked_tensor, combinations, alpha, gene_data, para_time):
    logger = init_logging("realdata02.log")
    dat = stacked_tensor[i,:,:]
    x = data[:, 0: int(shape[combinations[i][0]])] 
 
    y = data[:, int(shape[combinations[i][0]]): int(shape[combinations[i][0]] + shape[combinations[i][1]])] 
    z = data[:, int(shape[combinations[i][0]] + shape[combinations[i][1]]): data.shape[1]] 
    logger.info(x.shape)
    logger.info(y.shape)
    logger.info(z.shape)

    # n = 50
    # q=2
    # p=2
    # m=5
    # x = torch.randn(n,p) 
    # y = torch.randn(n,q)
    # z = torch.randn(n,m) 
    # logger.info(x.shape)
    # logger.info(y.shape)
    # logger.info(z.shape)

    torch.manual_seed((i + 1) *123)

    
    tasks_per_machine = para_time // 8
    tmp = tasks_per_machine * 8 
    adjusted_index = (i - tmp if i > (tmp-1) else i) // tasks_per_machine
    machine_index = adjusted_index % 8
    device = args.device+':'+str(machine_index)

    logger.info(f"任务 {i} 完成")  # 写入日志
    result = gauss_fun(i,x, y, z, device)
    return result


In [13]:
cuda_cores = 55 # GPU 核心数量
# cuda_cores = 100  # 可根据实际的 GPU 核心数量或 CPU 线程设置
n_my = 55
# sim_list = {'my_data_ex10':0}
sim_list = {'my_data_ex8': 0}
for sim, dep in sim_list.items():
    resmy = torch.zeros(n_my, 3, 3)
    print(sim, dep)
    try:
        with Parallel(n_jobs=cuda_cores) as parallel:
            tmp_results = parallel(
                delayed(run_simulation)(i, stacked_tensor, combinations, alpha, getattr(gene_data_Copy1, sim), cuda_cores)
                for i in tqdm(range(n_my))
            )
        results = torch.stack(tmp_results).to('cpu')
        del tmp_results
        
        print(results.shape)
        # results = torch.stack(Parallel(n_jobs=cuda_cores)(delayed(run_simulation)(i, n, p, q, m, dep, n_1, n_2, n_3, n_tilde, alpha, getattr(gene_data, sim), cuda_cores) for i in tqdm(range(n_my))))
        res_all = torch.randn(n_my, 4)
        res_size = torch.randn(n_my, 3)
        res_n = torch.randn(n_my, 3)
        for i in range(n_my):
            res_all[i, 0] = results[i, :, 3][1]
            res_all[i, 1:4] = results[i, :, 0]
            res_size[i, :] = results[i, :, 1]
            res_n[i, :] = results[i, :, 2]
            
        res_my = res_all
        
        df1 = pd.DataFrame(res_my)
    
        dic_path = './ind2_realdata'
        #file_path = os.path.join(dic_path, f'result_{sim}2.xlsx')
        
        # 将DataFrame保存到Excel文件
        # df1.to_excel(file_path, index=False, sheet_name='Sheet1')

        # 将 DataFrame 保存到 CSV 文件
        file_path = os.path.join(dic_path, 'result_data02.csv')
        df1.to_csv(file_path, index=False, encoding='utf-8')
        
        
        df2 = pd.DataFrame(res_size)
        # file_path2 = os.path.join(dic_path, f'result_{sim}power.xlsx')
        file_path2 = os.path.join(dic_path, 'result_data0size2.csv')
        # 将DataFrame保存到Excel文件
        # df2.to_excel(file_path2, index=False, sheet_name='Sheet1')
        df2.to_csv(file_path2, index=False, encoding='utf-8')
        
        df3 = pd.DataFrame(res_n)
        # file_path3 = os.path.join(dic_path, f'result_{sim}_n1.xlsx')
        file_path3 = os.path.join(dic_path, 'result_data02n1.csv')
        
        # 将DataFrame保存到Excel文件
        # df3.to_excel(file_path2, index=False, sheet_name='Sheet1')
        df3.to_csv(file_path3, index=False, encoding='utf-8')
        print(res_my)
        send_email(f"47号机{sim}的realr程序运行成功", str(res_my))

        del results
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    except Exception as e:
        # 捕获异常并发送邮件
        error_message = f"程序运行出错:\n{traceback.format_exc()}"
        print(error_message)
        send_email(f"47号机{sim}的real程序出错通知", error_message)

my_data_ex8 0


  0%|          | 0/55 [00:00<?, ?it/s]

2025-02-11 07:47:34,688 - torch.Size([755, 21])
2025-02-11 07:47:34,688 - torch.Size([755, 63])
2025-02-11 07:47:34,688 - torch.Size([755, 401])
2025-02-11 07:47:34,690 - 任务 0 完成
2025-02-11 07:47:35,024 - torch.Size([755, 21])
2025-02-11 07:47:35,025 - torch.Size([755, 30])
2025-02-11 07:47:35,025 - torch.Size([755, 434])
2025-02-11 07:47:35,027 - 任务 1 完成
2025-02-11 07:47:35,336 - torch.Size([755, 21])
2025-02-11 07:47:35,337 - torch.Size([755, 23])
2025-02-11 07:47:35,337 - torch.Size([755, 441])
2025-02-11 07:47:35,338 - 任务 2 完成
2025-02-11 07:47:35,979 - torch.Size([755, 21])
2025-02-11 07:47:35,979 - torch.Size([755, 58])
2025-02-11 07:47:35,979 - torch.Size([755, 406])
2025-02-11 07:47:35,980 - 任务 4 完成
2025-02-11 07:47:35,988 - torch.Size([755, 21])
2025-02-11 07:47:35,988 - torch.Size([755, 63])
2025-02-11 07:47:35,988 - torch.Size([755, 401])
2025-02-11 07:47:35,989 - 任务 3 完成
2025-02-11 07:47:36,340 - torch.Size([755, 21])
2025-02-11 07:47:36,340 - torch.Size([755, 73])
2025-02-1

755


2025-02-11 08:02:50,071 - 第0次的10网络完成
2025-02-11 08:02:52,210 - 第0次的50网络完成
2025-02-11 08:02:54,319 - 第0次的50网络完成
2025-02-11 08:02:54,965 - 第0次的70网络完成
2025-02-11 08:02:57,584 - 第0次的30网络完成
2025-02-11 08:02:58,081 - 第0次的50网络完成
2025-02-11 08:02:59,247 - 第0次的20网络完成
2025-02-11 08:02:59,867 - 第0次的90网络完成
2025-02-11 08:03:00,002 - 第0次的70网络完成
2025-02-11 08:03:00,864 - 第0次的20网络完成
2025-02-11 08:03:02,873 - 第0次的40网络完成
2025-02-11 08:03:03,893 - 第0次的110网络完成
2025-02-11 08:03:04,962 - 第0次的40网络完成
2025-02-11 08:03:05,800 - 第0次的80网络完成
2025-02-11 08:03:05,853 - 第0次的0网络完成
2025-02-11 08:03:06,658 - 第0次的70网络完成
2025-02-11 08:03:11,409 - 第0次的10网络完成
2025-02-11 08:03:12,674 - 第0次的40网络完成
2025-02-11 08:03:16,722 - 第0次的20网络完成
2025-02-11 08:03:18,697 - 第0次的60网络完成
2025-02-11 08:03:19,529 - 第0次的90网络完成
2025-02-11 08:03:19,551 - 第0次的50网络完成
2025-02-11 08:03:23,748 - 第0次的80网络完成
2025-02-11 08:03:24,462 - 第0次的90网络完成
2025-02-11 08:03:25,321 - 第0次的60网络完成
2025-02-11 08:03:26,783 - 第0次的30网络完成
2025-02-11 08:03:27,294 - 第0次的40网络完成
2

...

755
755


2025-02-11 08:25:52,233 - 第0次的130网络完成


755


2025-02-11 08:27:10,993 - 第0次的140网络完成


755
755
755
torch.Size([55, 3, 4])
tensor([[0.0000e+00, 1.6200e-02, 3.2600e-02, 2.0000e-02],
        [1.0000e+00, 1.5600e-02, 1.1820e-01, 3.1000e-02],
        [2.0000e+00, 1.8320e-01, 1.1360e-01, 4.4660e-01],
        [3.0000e+00, 1.9600e-02, 5.7000e-02, 5.1800e-02],
        [4.0000e+00, 8.4200e-02, 1.0280e-01, 2.9400e-02],
        [5.0000e+00, 3.1240e-01, 1.8800e-02, 7.8000e-03],
        [6.0000e+00, 2.3000e-01, 4.9600e-02, 3.0200e-02],
        [7.0000e+00, 6.3600e-02, 3.6600e-02, 3.9280e-01],
        [8.0000e+00, 3.4900e-01, 5.9400e-02, 2.9800e-02],
        [9.0000e+00, 2.5000e-02, 1.6640e-01, 1.1580e-01],
        [1.0000e+01, 4.7000e-02, 6.0000e-04, 2.0000e-04],
        [1.1000e+01, 6.2000e-03, 2.0000e-03, 2.0000e-02],
        [1.2000e+01, 0.0000e+00, 0.0000e+00, 6.0000e-04],
        [1.3000e+01, 0.0000e+00, 0.0000e+00, 4.0000e-04],
        [1.4000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.5000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.6000e+01, 2.0000e-04, 6.00

In [14]:
name1 = ["CS" , "CD", "CSt" , "Eng"  ,"Fin" ,"HC" , "Ind" , "IT" , "Mat" , "RE" , "Uti"]
combi_name1 = list(itertools.combinations(name1, 2))
logger = init_logging("realdata.log")  # 为每个任务初始化日志

shape = torch.zeros(11)

title_z = '68return_' + name1[0] + '.csv'
# z = torch.from_numpy(pd.read_csv(title_z))
data = torch.from_numpy(pd.read_csv(title_z).iloc[:, 1:].to_numpy())
shape[0] = data.shape[1]
for t in range(10):
        title_zz = '68return_' + name1[t+1] + '.csv'
        zz = torch.from_numpy(pd.read_csv(title_zz).iloc[:, 1:].to_numpy())
        shape[t+1] = zz.shape[1]
        data = torch.cat((data, zz), dim=1)

combinations = list(itertools.combinations(range(11), 2))
print(shape)


stacked_tensor = torch.zeros(55, data.shape[0], data.shape[1])
for i in range(55):
    mask = torch.ones(data.shape[1], dtype=torch.bool)
    index = combinations[i]
    # print(index)
    indx1 = int( torch.sum(shape[:index[0]]) )
    indx2 = int( torch.sum(shape[:index[0]+1]) )
    indy1 = int( torch.sum(shape[:index[1]]) )
    indy2 = int( torch.sum(shape[:index[1]+1]) )
    x = data[:, int(indx1):int(indx2)] 
    # print(x.shape)
    y = data[:, int(indy1):int(indy2)] 
    # print(y.shape)
    mask[indx1:indx2] = False  # 将 x 的范围设置为 False
    mask[indy1:indy2] = False  # 将 y 的范围设置为 False
    z = data[:, mask]  # 使用布尔掩码提取 z
    # print(z.shape)
    dat = torch.cat((x, y, z), dim=1)
    stacked_tensor[i] = dat
    # print(i)


tensor([21., 63., 30., 23., 63., 58., 73., 73., 23., 29., 29.])


In [15]:

def run_simulation(i, stacked_tensor, combinations, alpha, gene_data, para_time):
    logger = init_logging("realdata68.log")
    dat = stacked_tensor[i,:,:]
    x = data[:, 0: int(shape[combinations[i][0]])] 
 
    y = data[:, int(shape[combinations[i][0]]): int(shape[combinations[i][0]] + shape[combinations[i][1]])] 
    z = data[:, int(shape[combinations[i][0]] + shape[combinations[i][1]]): data.shape[1]] 
    logger.info(x.shape)
    logger.info(y.shape)
    logger.info(z.shape)

    # n = 50
    # q=2
    # p=2
    # m=5
    # x = torch.randn(n,p) 
    # y = torch.randn(n,q)
    # z = torch.randn(n,m) 
    # logger.info(x.shape)
    # logger.info(y.shape)
    # logger.info(z.shape)

    torch.manual_seed((i + 1) *123)

    
    tasks_per_machine = para_time // 8
    tmp = tasks_per_machine * 8 
    adjusted_index = (i - tmp if i > (tmp-1) else i) // tasks_per_machine
    machine_index = adjusted_index % 8
    device = args.device+':'+str(machine_index)

    logger.info(f"任务 {i} 完成")  # 写入日志
    result = gauss_fun(i,x, y, z, device)
    return result


In [16]:
cuda_cores = 55 # GPU 核心数量
# cuda_cores = 100  # 可根据实际的 GPU 核心数量或 CPU 线程设置
n_my = 55
# sim_list = {'my_data_ex10':0}
sim_list = {'my_data_ex8': 0}
for sim, dep in sim_list.items():
    resmy = torch.zeros(n_my, 3, 3)
    print(sim, dep)
    try:
        with Parallel(n_jobs=cuda_cores) as parallel:
            tmp_results = parallel(
                delayed(run_simulation)(i, stacked_tensor, combinations, alpha, getattr(gene_data_Copy1, sim), cuda_cores)
                for i in tqdm(range(n_my))
            )
        results = torch.stack(tmp_results).to('cpu')
        del tmp_results
        
        print(results.shape)
        # results = torch.stack(Parallel(n_jobs=cuda_cores)(delayed(run_simulation)(i, n, p, q, m, dep, n_1, n_2, n_3, n_tilde, alpha, getattr(gene_data, sim), cuda_cores) for i in tqdm(range(n_my))))
        res_all = torch.randn(n_my, 4)
        res_size = torch.randn(n_my, 3)
        res_n = torch.randn(n_my, 3)
        for i in range(n_my):
            res_all[i, 0] = results[i, :, 3][1]
            res_all[i, 1:4] = results[i, :, 0]
            res_size[i, :] = results[i, :, 1]
            res_n[i, :] = results[i, :, 2]
            
        res_my = res_all
        
        df1 = pd.DataFrame(res_my)
    
        dic_path = './ind2_realdata'
        #file_path = os.path.join(dic_path, f'result_{sim}2.xlsx')
        
        # 将DataFrame保存到Excel文件
        # df1.to_excel(file_path, index=False, sheet_name='Sheet1')

        # 将 DataFrame 保存到 CSV 文件
        file_path = os.path.join(dic_path, 'result_data68.csv')
        df1.to_csv(file_path, index=False, encoding='utf-8')
        
        
        df2 = pd.DataFrame(res_size)
        # file_path2 = os.path.join(dic_path, f'result_{sim}power.xlsx')
        file_path2 = os.path.join(dic_path, 'result_data68size2.csv')
        # 将DataFrame保存到Excel文件
        # df2.to_excel(file_path2, index=False, sheet_name='Sheet1')
        df2.to_csv(file_path2, index=False, encoding='utf-8')
        
        df3 = pd.DataFrame(res_n)
        # file_path3 = os.path.join(dic_path, f'result_{sim}_n1.xlsx')
        file_path3 = os.path.join(dic_path, 'result_data68n1.csv')
        
        # 将DataFrame保存到Excel文件
        # df3.to_excel(file_path2, index=False, sheet_name='Sheet1')
        df3.to_csv(file_path3, index=False, encoding='utf-8')
        print(res_my)
        send_email(f"47号机{sim}的real程序运行成功", str(res_my))

        del results
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    except Exception as e:
        # 捕获异常并发送邮件
        error_message = f"程序运行出错:\n{traceback.format_exc()}"
        print(error_message)
        send_email(f"47号机{sim}的real程序出错通知", error_message)

my_data_ex8 0


  0%|          | 0/55 [00:00<?, ?it/s]

2025-02-11 08:35:19,585 - torch.Size([753, 21])
2025-02-11 08:35:19,586 - torch.Size([753, 63])
2025-02-11 08:35:19,588 - torch.Size([753, 401])
2025-02-11 08:35:19,589 - 任务 0 完成
2025-02-11 08:35:22,578 - torch.Size([753, 21])
2025-02-11 08:35:22,578 - torch.Size([753, 30])
2025-02-11 08:35:22,578 - torch.Size([753, 434])
2025-02-11 08:35:22,579 - 任务 1 完成
2025-02-11 08:35:22,836 - torch.Size([753, 21])
2025-02-11 08:35:22,836 - torch.Size([753, 23])
2025-02-11 08:35:22,836 - torch.Size([753, 441])
2025-02-11 08:35:22,837 - 任务 2 完成
2025-02-11 08:35:23,516 - torch.Size([753, 21])
2025-02-11 08:35:23,518 - torch.Size([753, 63])
2025-02-11 08:35:23,518 - torch.Size([753, 401])
2025-02-11 08:35:23,520 - 任务 3 完成
2025-02-11 08:35:23,708 - torch.Size([753, 21])
2025-02-11 08:35:23,708 - torch.Size([753, 58])
2025-02-11 08:35:23,708 - torch.Size([753, 406])
2025-02-11 08:35:23,709 - 任务 4 完成
2025-02-11 08:35:24,068 - torch.Size([753, 21])
2025-02-11 08:35:24,069 - torch.Size([753, 73])
2025-02-1

753


2025-02-11 08:50:45,264 - 第0次的70网络完成
2025-02-11 08:50:46,917 - 第0次的0网络完成
2025-02-11 08:50:47,725 - 第0次的50网络完成
2025-02-11 08:50:49,011 - 第0次的0网络完成
2025-02-11 08:50:50,646 - 第0次的40网络完成
2025-02-11 08:50:53,818 - 第0次的60网络完成
2025-02-11 08:50:54,709 - 第0次的10网络完成
2025-02-11 08:50:56,280 - 第0次的50网络完成


...

753


2025-02-11 09:05:57,503 - 第0次的130网络完成


753


2025-02-11 09:06:32,898 - 第0次的100网络完成


753
753
753
753
753
753
753
753
753
753
torch.Size([55, 3, 4])
tensor([[0.0000e+00, 3.6000e-03, 4.8000e-03, 3.0000e-03],
        [1.0000e+00, 5.0780e-01, 1.2000e-03, 1.7800e-02],
        [2.0000e+00, 7.7800e-02, 4.8000e-03, 4.8000e-03],
        [3.0000e+00, 6.2000e-03, 1.1000e-02, 9.4000e-03],
        [4.0000e+00, 3.3200e-02, 2.4400e-02, 4.8440e-01],
        [5.0000e+00, 1.1080e-01, 2.7600e-02, 3.2540e-01],
        [6.0000e+00, 7.0000e-03, 1.9800e-02, 1.8400e-02],
        [7.0000e+00, 2.1580e-01, 8.0000e-04, 0.0000e+00],
        [8.0000e+00, 3.7120e-01, 2.5160e-01, 1.2780e-01],
        [9.0000e+00, 1.2000e-03, 3.9800e-02, 1.7200e-02],
        [1.0000e+01, 0.0000e+00, 2.6000e-02, 0.0000e+00],
        [1.1000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.2000e+01, 0.0000e+00, 9.4200e-02, 1.1220e-01],
        [1.3000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.4000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.5000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [